In [91]:
import json 
import os

def load_json(filename):
    path = os.path.join(os.getcwd(), filename)
    with open(path, 'r') as f:
        data = json.load(f)
    return data

answers = load_json("/gg2013answers.json")

NameError: name '__file__' is not defined